In [1]:
import fwdpy11
import numpy as np
import pandas as pd
import gzip
from collections import Counter

In [2]:
pop = fwdpy11.DiploidPopulation.load_from_file('results/simulations/sim-seed-1-pop.bin')

In [3]:
ts = pop.dump_tables_to_tskit()
ts

In [4]:
mut_clas = []
for m in ts.mutations():
    mut_clas.append(m.metadata['label'])
    
Counter(mut_clas)


Counter({0: 686, 1: 15, 2: 4, 3: 4})

In [17]:
m

Mutation(id=708, site=708, node=14877, derived_state='1', parent=-1, metadata={'h': 1.0, 'key': 419, 'label': 0, 'neutral': True, 'origin': 49994, 's': 0.0}, time=6.0)

In [27]:
# WHAT ARE THE SELECTION COEFFICIENTS FOR THE SELECTED MUTATIONS?
for m in ts.mutations():
    if not m.metadata['neutral']:
        if m.metadata['label'] == 1: # 1 is for missense
            print(m.metadata['s'])

0.1707705883578485
0.08391221548694464
4.629691481511648e-07
0.031745806348085034
0.058623981428570476
3.550835686710066e-05
0.07720343456897279
3.0582206333096505e-09
0.2351499694651107
3.9779648916179214e-08
3.440033661138962e-09
1.6456497031764145e-05
0.13185312091923135
0.0016686845926068275
0.0061645140296173375


In [5]:
nodes = np.array(pop.tables.nodes, copy=False)
alive_nodes = pop.alive_nodes
deme0_nodes = alive_nodes[np.where(nodes["deme"][alive_nodes] == 0)[0]]


In [6]:
# SFS (neutral + selected mutations)
pop.tables.fs([deme0_nodes[:100]], include_neutral=True)

masked_array(data=[--, 44, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 2, 1, --],
             mask=[ True, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, 

In [7]:
def get_sfs(pop, n, include_neutral):
    nodes = np.array(pop.tables.nodes, copy=False)
    alive_nodes = pop.alive_nodes
    deme0_nodes = alive_nodes[np.where(nodes["deme"][alive_nodes] == 0)[0]]
    # SFS (neutral + selected mutations)
    return pop.tables.fs([deme0_nodes[:n]], include_neutral=include_neutral).data

In [8]:
x = get_sfs(pop, 100, True)

In [9]:
def load_pop(sim_id):
    inf = f'results/simulations/sim-seed-{sim_id}-pop.bin'
    return fwdpy11.DiploidPopulation.load_from_file(inf)

In [16]:
pops = [load_pop(i) for i in range(1, 51)]

sfss_neutral = [get_sfs(p, 100, True) for p in pops]
sfss_neutral = np.sum(sfss_neutral, axis=0)

sfss_nonneutral = [get_sfs(p, 100, False) for p in pops]
sfss_nonneutral = np.sum(sfss_nonneutral, axis=0)

In [17]:
derived_freq = range(0, len(sfss_neutral))


sfs = pd.DataFrame(
{
    'Derived_Freq': derived_freq
})

In [18]:
sfs['All_SNPs_freq'] = sfss_neutral
sfs['Selected_freq'] = sfss_nonneutral

In [19]:
sfs.to_csv('results/SFSs_convoluted.csv', index=False)

In [20]:
sfss_neutral

array([29173,  1870,   252,    86,    31,    17,     8,    18,     8,
           8,     6,     4,     5,     4,     2,     2,     1,     1,
           2,     1,     3,     2,     3,     2,     0,     0,     0,
           2,     2,     2,     0,     1,     1,     2,     0,     0,
           0,     2,     0,     1,     1,     5,     0,     1,     0,
           0,     0,     0,     0,     0,     0,     0,     1,     2,
           1,     1,     0,     1,     0,     1,     0,     1,     0,
           1,     0,     0,     0,     1,     2,     4,     0,     3,
           2,     2,     1,     1,     0,     0,     1,     2,     3,
           2,     2,     1,     1,     3,     0,     2,     2,     0,
           3,     5,     1,     2,     1,     4,     5,     6,     4,
          12,    47])

In [21]:
sfss_nonneutral

array([659,  62,  17,   5,   5,   3,   2,   4,   3,   2,   3,   0,   2,
         1,   1,   0,   1,   0,   1,   0,   1,   1,   1,   1,   0,   0,
         0,   2,   0,   1,   0,   0,   1,   1,   0,   0,   0,   1,   0,
         0,   1,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   1,   1,   1,   0,   0,   0,   0,   0,   1,   0,   1,   0,
         0,   0,   1,   2,   2,   0,   2,   1,   2,   0,   0,   0,   0,
         1,   0,   1,   0,   0,   1,   1,   1,   0,   1,   0,   0,   1,
         2,   0,   0,   1,   1,   1,   6,   1,   7,  26])